In [1]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=5

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    return c'*v
end
# random quadratic objective function f
v=reverse(monomials(x,0:2))
f=generate_random_poly(v)


# unit sphere constraint
m=n
q=floor(Int64,n/m)
R=ones(Float64,m)./n
T=[(j-1)*q+1:j*q for j in 1:m-1]
append!(T,[(m-1)*q+1:n])

g=[R[j]-sum(x[T[j]].^2) for j in 1:m]

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=0#ceil(Int64, n/7)

h=Vector{Polynomial{true,Float64}}(undef,l)
randx=[2*rand(length(T[j])).-1 for j in 1:m]# create a feasible solution
randx=[sqrt(R[j])*rand(1)[1]*randx[j]/norm(randx[j]) for j in 1:m]
randx=vcat(randx...)

for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=5
Number of inequality constraints: m=5
Number of equality constraints: l=0


In [2]:
using ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=false);

┌ Info: Precompiling ctpPOP [8dd00c7b-d197-4c04-bcb1-2ef5c20236b2]
└ @ Base loading.jl:1273
┌ Warning: Package ctpPOP does not have LightGraphs in its dependencies:
│ - If you have ctpPOP checked out for development and have
│   added LightGraphs as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ctpPOP
└ Loading LightGraphs into ctpPOP from project dependency, future warnings for ctpPOP are suppressed.


In [3]:
k=2
println("Relaxed order: k=",k)

Relaxed order: k=2


In [4]:
using ctpPOP

opt_val1=ctpPOP.POP_dense_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                         maxit=Int64(1e6),tol=1e-3,
                         use_eqcons_to_get_constant_trace=false,
                         check_tol_each_iter=true)

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer, max_cpu_time=60.0)` becomes `optimizer_with_attributes(Ipopt.Optimizer, "max_cpu_time" => 60.0)`.
│   caller = ip:0x0
└ @ Core :-1


  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=6
  Size of the largest block: s^max=21
  Number of equality trace constraints: zeta=211
Modeling time:
 15.773055 seconds (49.72 M allocations: 2.441 GiB, 6.48% gc time)
iter=1   val=-3.976297117223254   gap=0.0009240987698501636   feas=3.3906376019915965
iter=2   val=-1.660149868275019   gap=0.5705762781135936   feas=0.8159604901765637
iter=4   val=-1.4536005148985913   gap=0.42715922172136234   feas=0.8565159675754054
iter=8   val=-1.186612461800377   gap=0.285824863440713   feas=0.34604887996312667
iter=16   val=-1.1870602374679795   gap=0.03504571633975418   feas=0.22083295615884724
iter=32   val=-1.1144347625049458   gap=0.06673682906569083   feas=0.10709339999338986
iter=64   val=-1.13869735482517   gap=0.13964217447441887   feas=0.026205226809281976
iter=128   val=-1.1704704445526133   gap=0.007520649609154053   feas=0.014456837377760844
iter=256   val=-1.1909757996145913   gap=0.003

-1.2036023038987582

In [5]:
using ctpPOP

opt_val2=ctpPOP.POP_dense_SOS(x,f,g,h,k,tol=1e-3)

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 126             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 6               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimizat

-1.205702223871009

In [6]:
using ctpPOP

opt_val=ctpPOP.POP_dense_LMBM(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                         tol=1e-3,use_eqcons_to_get_constant_trace=false)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=6
  Size of the largest block: s^max=21
  Number of equality trace constraints: zeta=211
Modeling time:
  0.005135 seconds (15.08 k allocations: 1.646 MiB)
---------------
| Parameters: |
---------------
n:       211
maxtime: 300000.000000
na:      2
mcu:     5
mc:      7
rpar: 
ipar: 
 Entry to LMBM:
 NIT=    1  NFE=    1  F= 0.59723186D+00  WK= 0.6071D+00  QK= 0.3036D+00
 NIT=    2  NFE=    2  F= 0.48241043D+00  WK= 0.2593D+00  QK= 0.2176D+00
 NIT=    3  NFE=    3  F= 0.39247071D+00  WK= 0.8683D-01  QK= 0.6816D-01
 NIT=    4  NFE=    4  F= 0.30955238D+00  WK= 0.1124D+01  QK= 0.5782D-01
 NIT=    5  NFE=    5  F= 0.30955238D+00  WK= 0.1156D+00  QK= 0.5782D-01
 NIT=    6  NFE=    6  F= 0.21898998D+00  WK= 0.8340D-01  QK= 0.1726D+00
 NIT=    7  NFE=    7  F= 0.21898998D+00  WK= 0.3152D-01  QK= 0.2506D-01
 NIT=    8  NFE=    8  F= 0.21024023D+00  WK= 0.3829D-01  QK= 0.1297D+00
 NIT=    9  NFE= 

-1.2107334946190855

In [ ]:
using ctpPOP

optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)